# Instructons to use

### To load and initate relationships between the 18 orgs downloaded Run the cells below the markdown cells

Run cell

In [ ]:
## Import neo4j.v1 to interface with neo4j database and jsonn to read and manipulate the json files
from neo4j.v1 import GraphDatabase, basic_auth
import json

Run cell

In [ ]:
## establishes the connection to the neo4j database to use 

driver = GraphDatabase.driver("/Users/jgossess/Documents/Neo4j/gitScrape_demo1", auth=basic_auth("neo4J", "test"))
# driver = GraphDatabase.driver("<insert path to database>", auth=basic_auth("<insert username>", "<insert password>"))

In [ ]:
## read in the json file 
data = ''
with open("scraped/nasa.json", 'r') as jsonf:
    data = jsonf.read()

In [ ]:
## load the json file into a json object
json_data = json.loads(data)

In [ ]:
## create nodes for each file, explicit dependency, org, and repo in the neo4j database
lod = 0
pss = 0
## This statement creates a node for an org if the node does not yet exist 
create_org = "MERGE (o:org {name: {org}, org_link: {org_link}})"
## This statement creates a node for a dependency if the node does not yet exist
create_dep = "MERGE (d:dependency {name: {dep}})"
## This statement creates a node for a repo if the node does not yet exist
create_repo = "MERGE (r:repo {name: {repo}, repo_link: {repo_link}, org: {org}})"
## This statement creates a node for a file if the node does not yet exist
create_file = "MERGE (a:file {org: {org}, org_link: {org_link}, repo: {repo}, repo_link: {repo_link}, filename: {filename}, explicit_dep: {explicit_dep}, implicit_dep: {implicit_dep}})"

## This goes through the entries in the json and will creates nodes for all the new orgs, files, repos, and dependencies
for entry in range(len(json_data)):
    ## if there is a failure in the upload it will print the exception as well as the jsonfor the problem information
    try:
        prop = json_data[entry]
        prop['explicit_dep'] = [x for x in prop['explicit_dep'] if x != None]
        with driver.session() as session:
            session.run(create_org, prop)
            session.run(create_repo, prop)
            session.run(create_file, prop)
            for dep in prop['explicit_dep']:
                dep = {'dep': dep}
                session.run(create_dep, dep)
            session.sync()
        lod+=1
    except Exception as e:
        print(str(e))
        print(entry)
        pss+=1
        pass
print("loaded: "+str(lod))
print('skipped: '+str(pss))

# Use this to load all repos into neo4j

Run cell

In [ ]:
def load_neo4j(json_data):
    """Uploads the passed json data to neo4j database"""
    lod = 0
    pss = 0
    ## Statement to create org node if it does not exist
    create_org = "MERGE (o:org {name: {org}, org_link: {org_link}})"
    ## Statement to create dependency node if it does not exist
    create_dep = "MERGE (d:dependency {name: {dep}})"
    ## Statement to create repo node if it does not exist
    create_repo = "MERGE (r:repo {name: {repo}, repo_link: {repo_link}, org: {org}})"
    ## Statement to create file node if it does not exist
    create_file = "MERGE (a:file {org: {org}, org_link: {org_link}, repo: {repo}, repo_link: {repo_link}, filename: {filename}, explicit_dep: {explicit_dep}, implicit_dep: {implicit_dep}})"
    ## Go through json and create nodes for all new files, orgs, repos, and dependencies
    for entry in range(len(json_data)):
        try:
            prop = json_data[entry]
            prop['explicit_dep'] = [x for x in prop['explicit_dep'] if x != None]
            with driver.session() as session:
                session.run(create_org, prop)
                session.run(create_repo, prop)
                session.run(create_file, prop)
                for dep in prop['explicit_dep']:
                    dep = {'dep': dep}
                    session.run(create_dep, dep)
                session.sync()
            lod+=1
        except Exception as e:
            print(str(e))
            print(entry)
            pss+=1
            pass
    print("loaded: "+str(lod))
    print('skipped: '+str(pss))

Run cell

In [ ]:
def create_relate():
    """Creates relationships between all the nodes"""
    ## Create from relationship between files and the repos
    repo_file_relate = 'MATCH (re:repo),(f:file) WHERE re.name = f.repo and re.repo_link = f.repo_link MERGE (f)-[r:from]->(re) RETURN r'
    ## Create from relationship between repos and orgs
    org_repo_relate = 'MATCH (o:org),(r:repo) WHERE o.name = r.org MERGE (r)-[f:from]->(o) RETURN f'
    ## Create uses relationship between dependencies and files
    dep_file_relate = 'MATCH (d:dependency),(f:file) WHERE d.name in f.explicit_dep MERGE (f)-[r:uses]->(d) RETURN r'
    with driver.session() as session:
        session.run(repo_file_relate)
        session.run(org_repo_relate)
        session.run(dep_file_relate)

Run cell

In [ ]:
## Relitive path to the json data as seen in the github file structure for this repo
json_paths = ["scraped/18f.json","scraped/NASA-rdt.json","scraped/arcticlcc.json",
              "scraped/ccmc.json","scraped/cfpb.json","scraped/cmsgov.json","scraped/commercedataservice.json",
              "scraped/defense-cyber-crime-center.json","scraped/department-of-veterans-affairs.json",
              "scraped/dhs-ncats.json","scraped/energyapps.json","scraped/eregs.json","scraped/fcc.json",
              "scraped/fda.json","scraped/nasa-develop.json","scraped/nasa.json","scraped/nasa-gibs.json","scraped/nasa.json"]

Run cell

In [ ]:
## Go through all the org jsons and load into the neo4j database as well as creating the relationships
for path in json_paths:
    data = ''
    with open(path, 'r') as jsonf:
        data = jsonf.read()
    jdata = json.loads(data)
    load_neo4j(jdata)
create_relate()